## CHEME 5660 Practice Prelim 2 Options Question

A trader at Olin Financial, an up-and-coming hedge fund, sold a short strangle on firm XYZ with a 12/16 expiration. 

__Assumptions__: (i) the short put (contract 1) has a strike price of $K_{1}$ = 50.0 USD/share and an implied volatility of 50.79\%;
(ii) the short call (contract 2) has a strike price of $K_{2}$ = 57.50 USD/share and an implied volatility of 46.79\%;
(iii) there are 49 days to 12/16; (iv) the current share price of XYZ is 53.60 USD/share; (v) the risk-free rate is 4.05\%.

Use the Jupyter notebook `CHEME-5660-PP2-Options.ipynb`, and any associated data sets, to answer the following questions:

* a) Compute the premiums for the put $\mathcal{P}_{1}$ and call $\mathcal{P}_{2}$ contracts for the 12/16 short strangle on firm XYZ using the Cox, Ross, and Rubinstein (CRR) binomial lattice model. 
* b) Compute the maximum profit and break-even points for the Olin Financial short strangle position on XYZ.
* c) Compute the probability of the profit at expiration for the short strangle position using the share price distribution from the `Equity` notebook. 

## Solution

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/prelims/P2/practice`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/prelims/P2/practice/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/prelims/P2/practice/Manifest.toml`


In [2]:
# load external packages that are required for the calculations -
using DataFrames
using CSV
using Dates
using Statistics
using LinearAlgebra
using Plots
using Colors
using Distributions
using StatsPlots
using PQEcolaPoint

# setup paths to load XYZ OHLC data set -
const _NOTEBOOK_ROOT = pwd();
const _PATH_TO_DATA = joinpath(_NOTEBOOK_ROOT, "data");

In [3]:
include("CHEME-5560-PP2-CodeLib.jl");

In [4]:
# Date -
D = Date(2022, 12, 16); # date when contracts expire 

# contract 1 parameters -
IV₁ = 50.79; # implied volatility for contract 1 (short put)
K₁ = 50.0;   # strike price short put
T₁ = ticker("P", "XYZ", D, K₁);

# contract 2 parameters -
IV₂ = 46.79; # implied volatility for contract 2 (short call)
K₂ = 57.50;  # strike price short call
T₂ = ticker("C", "XYZ", D, K₂);

# setup some shared constants
B = 365.0;   # number of days in a year
μ = 0.0405;  # risk-free rate
DTE = 49.0;  # days to expiration

# What is the current share price?
Sₒ = 53.60;

# How many levels on the tree?
L = 100;

### a) Estimate the price of the call and put contracts

In [5]:
# build the contracts 
put_contract_model = build(PutContractModel, (
    ticker = T₁,
    expiration_date = D,
    strike_price = K₁,
    premium = 0.0,
    number_of_contracts = 1,
    direction = 1, # Bug or feature? => always use 1 here (even if we are selling)
    current_price = 0.0      
));

call_contract_model = build(CallContractModel, (
    ticker = T₂,
    expiration_date = D,
    strike_price = K₂,
    premium = 0.0,
    number_of_contracts = 1,
    direction = 1, # Bug or feature? => always use 1 here (even if we are selling)
    current_price = 0.0      
));

In [6]:
# build the lattice models -
lattice_model_put = build(CRRLatticeModel; number_of_levels=(L+1), Sₒ = Sₒ, σ = (IV₁/100.0), μ = μ, T = (DTE/B));
lattice_model_call = build(CRRLatticeModel; number_of_levels=(L+1), Sₒ = Sₒ, σ = (IV₂/100.0), μ = μ, T = (DTE/B));

In [7]:
# compute the price of the put contract -
price_put_contract = premium(put_contract_model, lattice_model_put);
println("Put contract price = $(price_put_contract) (USD/share)")

Put contract price = 2.1925398489525816 (USD/share)


In [8]:
# compute the price of the call contract -
price_call_contract = premium(call_contract_model, lattice_model_call);
println("Call contract price = $(price_call_contract) (USD/share)")

Call contract price = 2.2322342690115713 (USD/share)


### b) Compute the maximum profit and breakeven points for the short strangle at expiration

In [9]:
# max profit occurs when trade is opened -
max_profit = (price_put_contract + price_call_contract)
println("Max profit = $(max_profit) (USD/share)")

Max profit = 4.424774117964153 (USD/share)


In [10]:
# Break even 1 (low) -
B₁ = K₁ - (price_call_contract + price_put_contract)
println("Break-even share price (low) B₁ = $(B₁) (USD/share)")

Break-even share price (low) B₁ = 45.575225882035845 (USD/share)


In [11]:
# Break even 2 (high) -
B₂ = K₂ + (price_call_contract + price_put_contract)
println("Break-even share price (high) B₂= $(B₂) (USD/share)")

Break-even share price (high) B₂= 61.924774117964155 (USD/share)


### c) Compute the probability of profit at expiration 

In [12]:
# build a log normal distribution from the previous GBM simulations 
μ̂ = 3.823
σ̂ = 0.1846
d = LogNormal(μ̂, σ̂);

In [13]:
# sample the distribution -
N = 10000;
S_expiration = rand(d,N);

In [14]:
# compute the probability of profit -

# we are profitable if we close between the break even prices -
prob_less_than_B1 = P(S_expiration, B₁);
prob_less_than_B2 = P(S_expiration, B₂);

In [15]:
POP = prob_less_than_B2 - prob_less_than_B1

0.45949999999999996